# Séance 1 :  Deep Learning - Introduction à Pytorch 

Les notebooks sont très largement inspirés des cours de **N. Baskiotis et B. Piwowarski**. Ils peuvent être complétés efficacement par les tutoriels *officiels* présents sur le site de pytorch:
https://pytorch.org/tutorials/

Au niveau de la configuration, toutes les installations doivent fonctionner sur Linux et Mac. Pour windows, ça peut marcher avec Anaconda à jour... Mais il est difficile de récupérer les problèmes.

* Aide à la configuration des machines: [lien](https://dac.lip6.fr/master/environnement-deep/)
* Alternative 1 à Windows: installer Ubuntu sous Windows:  [Ubuntu WSL](https://ubuntu.com/wsl)
* Alternative 2: travailler sur Google Colab (il faut un compte gmail + prendre le temps de comprendre comment accéder à des fichers) [Colab](https://colab.research.google.com)

In [ ]:
import torch
print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())

# pour les possesseurs de mac M1 avec la dernière version de pytorch:
print("Le calcul GPU est disponible ? ", torch.backends.mps.is_available())

import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [ ]:
## Chargement des données housing (depuis sklearn) et transformation en tensor.
# from sklearn.datasets import load_housing # => removed
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(data_home="./data/") ## chargement des données

housing_x = torch.tensor(housing['data'],dtype=torch.float) # penser à typer les données pour éliminer les incertitudes
housing_y = torch.tensor(housing['target'],dtype=torch.float)

print("Nombre d'exemples : ",housing_x.size(0), "Dimension : ",housing_x.size(1))
print("Nom des attributs : ", ", ".join(housing['feature_names']))

print(housing_x[:5])


## C. Architecture modulaires & réseaux de neurones
Dans le framework pytorch (et dans la plupart des frameworks analogues), le module est la brique de base qui permet de construire un réseau de neurones.  Il permet de représenter en particulier :
* une couche du réseau (linéaire : **torch.nn.Linear**, convolution : **torch.nn.convXd**, ...)
* une fonction d'activation (tanh : **torch.nn.Tanh**, sigmoïde : **torch.nn.Sigmoid** , ReLu : **torch.nn.ReLu**, ...)
* une fonction de coût (MSE : **torch.nn.MSELoss**, L1 :  **torch.nn.L1Loss**, CrossEntropy binaire: **torch.BCE**, CrossEntropy : **torch.nn.CrossEntropyLoss**, ...)
* mais également des outils de régularisation (BatchNorm : **torch.nn.BatchNorm1d**, Dropout : **torch.nn.Dropout**, ...)
* un ensemble de modules : en termes informatique, un module est un conteneur abstrait qui peut contenir d'autres conteneurs) : plusieurs modules peuvent être mis ensemble afin de former un nouveau module plus complexe.


Le fonctionnement est très proche des fonctions que nous avons vu ci-dessus (un module encapsule en fait une fonction de **torch.nn.Function**), mais de manière à gérer automatiquement les paramètres à apprendre. Un module est ainsi muni :
* d'une méthode **forward** qui permet de calculer la sortie du module à partir des entrées
* d'une méthode **backward** qui permet d'effectuer la rétro-propagation (localement).
* tous les paramètres sont automatiquement ajoutés dans une liste interne, accessible par la fonction **.parameters()** du module.

Ci-dessous un exemple de régression linéaire en utilisant les modules.


In [ ]:
EPOCHS=10
EPS = 1e-7

Xdim = housing_x.size(1)
## Création d'une couche linéaire de dimension Xdim->1
net = torch.nn.Linear(Xdim, 1) 

## Passe forward du module :  équivalent à net.forward(x)[:10]
print("Sortie du réseau", net(housing_x)[:10])
## affiche la liste des paramètres du modèle
print("Paramètres et noms des paramètres", list(zip(list(net.parameters()), list(net.named_parameters()))))


In [ ]:

## Création d'une fonction de loss aux moindres carrés
mseloss = torch.nn.MSELoss()
## on créé un optimiseur pour le réseau (paramètres w et b), avec un pas de gradient lr
optim = torch.optim.SGD(params=net.parameters(),lr=EPS) 
# Juste pour info, ce n'est pas utile, les paramètres sont déjà initialisés.
net.reset_parameters()

for i in range(EPOCHS):
    loss = mseloss(net(housing_x).view(-1,1),housing_y.view(-1,1))
    print(f"iteration : {i}, loss : {loss}")
    optim.zero_grad()
    loss.backward()
    optim.step()  

## C.1. Création d'un réseau de neurones

Avec ces briques élémentaires, il est très facile de définir un réseau de neurones standard :
* soit en utilisant le conteneur **torch.nn.Sequential** qui permet d'enchaîner séquentiellement plusieurs modules
* soit en définissant à la main un nouveau module.

Ci-dessous un exemple  pour créer un réseau à deux couches linéaires avec une fonction d'activation tanh des deux manières différentes. Vous remarquez qu'il n'y a pas besoin de définir la méthode **backward**, celle-ci est héritée du conteneur abstrait et ne fait qu'appeler séquentiellement en ordre inverse les méthodes **backward** des différents modules. 

In [ ]:
EPS = 1e-2
EPOCHS=50

#Réseau à la main (on le refera à la main derriere)
class DeuxCouches(torch.nn.Module):
  def __init__(self):
    super(DeuxCouches,self).__init__()
    self.un = torch.nn.Linear(Xdim,5)
    self.act = torch.nn.Tanh()
    self.deux = torch.nn.Linear(5,1)
  def forward(self,x):
    return self.deux(self.act(self.un(x)))

netDeuxCouches = DeuxCouches()

mseloss = torch.nn.MSELoss()
    
optim = torch.optim.SGD(params=netDeuxCouches.parameters(),lr=EPS)
for i in range(EPOCHS):
    loss = mseloss(netDeuxCouches(housing_x),housing_y.view(-1,1))
    print(f"iteration : {i}, loss : {loss}")
    optim.zero_grad()
    loss.backward()
    optim.step()
    

In [ ]:
#Création d'un réseau à 1 couche cachée avec le module séquentiel (remplace l'objet précédent)
netSeq = torch.nn.Sequential(torch.nn.Linear(Xdim,5),torch.nn.Tanh(),torch.nn.Linear(5,1))

mseloss = torch.nn.MSELoss()
    
optim = torch.optim.SGD(params=netSeq.parameters(),lr=EPS) # extraction auto des paramètres :)
for i in range(EPOCHS):
    loss = mseloss(netSeq(housing_x),housing_y.view(-1,1))
    print(f"iteration : {i}, loss : {loss}")
    optim.zero_grad()
    loss.backward()
    optim.step()

##  C.2. Méthodologie expérimentale et boîte à outils
Pytorch dispose d'un ensemble d'outils qui permettent de simplifier les démarches expérimentales. Nous allons voir en particulier : 
* le DataLoader qui permet de gérer le chargement de données, le partitionement et la constitution d'ensembles de test et d'apprentissage; 
* le checkpointing qui permet de sauvegarder/charger les modèles en cours d'entraînement.
* le TensorBoard (qui vient de tensorflow) qui permet de suivre l'évolution en apprentissage de vos modèles.


### C.2.1 DataLoader
Le <a href=https://pytorch.org/docs/stable/data.html>**DataLoader**</a> et la classe associée <a href=https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset> **Dataset**</a>  permettent en particulier de :
* charger des données
* pré-processer les données
* de gérer les mini-batchs (sous-ensembles sur lequel on effectue une descente de gradient).

La classe **Dataset** est une classe abstraite qui nécessite l'implémentation que d'une seule méthode, ```__getitem__(self,index)``` : elle renvoie le i-ème objet du jeu de données (généralement un couple *(exemple,label)*. 

La classe **TensorDataset** est l'instanciation la plus courante d'un **Dataset**, elle permet de créer un objet **Dataset** à partir d'une liste de tenseurs qui renvoie pour un index $i$ donné le tuple contenant les $i$-èmes ligne de chaque tenseur.

La classe **DataLoader** permet essentiellement de randomiser et de constituer des mini-batchs de façon simple à partir d'une instance de **Dataset**. Chaque mini-batch est constitué d'exemples tirés aléatoirement dans le **Dataset** passé en paramètre et mis bout à bout dans des tenseurs. La méthode ```collate_fn(*args)``` est utilisée pour cela (nous verrons une customization de cette fonction dans une séance ultérieure). C'est ce générateur qui est généralement parcouru lors de l'apprentissage à chaque itération d'optimisation.

Voici un exemple de code pour utiliser le DataLoader : 


In [ ]:
from torch.utils.data import DataLoader,TensorDataset

## Création d'un dataset à partir des deux tenseurs d'exemples et de labels
train_data = TensorDataset(housing_x,housing_y)
## On peut indexer et connaitre la longueur d'un dataset
print("DATASET:\n",len(train_data),train_data[5])

## Création d'un DataLoader
## tailles de mini-batch de 16, shuffle=True permet de mélanger les exemples
# loader est un itérateur sur les mini-batchs des données
loader = DataLoader(train_data, batch_size=16,shuffle=True ) # n'hésitez pas à jouer avec les paramètres

#Premier batch (aléatoire) du dataloader : (nb batch = len/batch_size)
print("DATA LOADER:\n",len(iter(loader)),"\n",next(iter(loader))[0].size())


In [ ]:

EPS=1e-4
netSeq = torch.nn.Sequential(torch.nn.Linear(Xdim,5),torch.nn.Tanh(),torch.nn.Linear(5,1))
optim = torch.optim.SGD(params=netSeq.parameters(),lr=EPS)

# La boucle d'apprentissage :
for i in range(EPOCHS):
    cumloss = 0
    # On parcourt tous les exemples par batch de 16 (paramètre batch_size de DataLoader)
    for bx,by in loader:
        loss = mseloss(netSeq(bx).view(-1),by)
        optim.zero_grad()
        loss.backward()
        optim.step()
        cumloss += loss.item() # item pour un scalaire (sinon .data ou detach)
    print(f"iteration : {i}, loss : {cumloss/len(loader)}") # loss sur un batch => diviser pour avoir une grandeur interprétable

### C.2.2 Checkpointing
Les modèles Deep sont généralement long à apprendre. Afin de ne pas perdre des résultats en cours de calcul, il est fortement recommander de faire du **checkpointing**, c'est-à-dire d'enregistrer des points d'étapes du modèle en cours d'apprentissage pour pouvoir reprendre à n'importe quel moment l'apprentissage du modèle en cas de problème.  Il s'agit en pratique de sauvegarder l'état du modèle et de l'optimisateur (et de tout autre objet qui peut servir lors de l'apprentissage) toutes les n itérations. Toutes les variables d'intérêt sont en général disponibles par la méthode **state_dict()** des modèles et de l'optimiseur. 

En pratique, vous pouvez utilisé un code dérivé de celui ci-dessous.




In [ ]:
# Il existe différentes solutions: en voici une
# mais ça marche
# 
import os

def save_state(epoch,model,optim,fichier):
      """ sauvegarde du modèle et de l'état de l'optimiseur dans fichier """
      state = {'epoch' : epoch, 'model_state': model.state_dict(), 'optim_state': optim.state_dict()}
      torch.save(state,fichier) # pas besoin de passer par pickle
 
def load_state(fichier,model,optim):
      """ Si le fichier existe, on charge le modèle et l'optimiseur """
      epoch = 0
      if os.path.isfile(fichier):
          state = torch.load(fichier)
          model.load_state_dict(state['model_state'])
          optim.load_state_dict(state['optim_state'])
          epoch = state['epoch']
      return epoch
 
netSeq = torch.nn.Sequential(torch.nn.Linear(Xdim,5),torch.nn.Tanh(),torch.nn.Linear(5,1))
optim = torch.optim.SGD(params=netSeq.parameters(),lr=EPS) # extraction auto des paramètres
fichier = "/tmp/netSeq.pth"
start_epoch = load_state(fichier,netSeq,optim)
for epoch in range(start_epoch,EPOCHS):
    cumloss = 0
    for bx,by in loader:
        loss = mseloss(netSeq(bx).view(-1),by)
        optim.zero_grad()
        loss.backward()
        optim.step()
        cumloss += loss.item()
    if epoch % 10 ==0: save_state(epoch,netSeq,optim,fichier)




### C.2.3 GPU 
Afin d'utiliser un GPU lors des calculs, il est nécessaire de transférer les données et le modèle sur le GPU par l'intermédiaire de la fonction **to(device)** des tenseurs et des modules.  Il est impossible de faire une opération lorsqu'une partie des tenseurs sont sur GPU et l'autre sur CPU. Il faut que tous les tenseurs et paramètres soient sur le même device ! On doit donc s'assurer que le modèle, les exemples et les labels sont sur GPU pour faire les opérations.

Par ailleurs, on peut connaître le device sur lequel est chargé un tenseur par l'intermédiaire de ```.device``` (mais pas pour un modèle, il faut aller voir les paramètres dans ce cas).

Une manière simple d'utiliser un GPU quand il existe et donc d'avoir un code agnostique est la suivante : 


In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# poru les possesseur de mac M1:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

## On charge le modèle sur GPU
## A faire avant la déclaration de l'optimiseur, sinon les paramètres optimisés ne seront pas les mêmes! 
## model =  model.to(device) 
loader = DataLoader(TensorDataset(housing_x,housing_y), batch_size=16,shuffle=True ) 

netSeq = torch.nn.Sequential(torch.nn.Linear(Xdim,5),torch.nn.Tanh(),torch.nn.Linear(5,1))
netSeq = netSeq.to(device)
optim = torch.optim.SGD(params=netSeq.parameters(),lr=EPS)

for i,(bx,by) in enumerate(loader):
    ## On charge le batch sur GPU
    bx, by = bx.to(device), by.to(device)
    loss = mseloss(netSeq(bx).view(-1),by)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i % 10 ==0: print("batch ",i)


print("Device du mini-batch : ", bx.device)



### C.2.4 TensorBoard

Durant l'apprentissage de vos modèles, il est agréable de visualiser de quelle manière évolue le coût, la précision sur l'ensemble de validation ainsi que d'autres éléments. TensorFlow dispose d'un outil très apprécié, le TensorBoard, qui permet de gérer très facilement de tels affichages. On retrouve tensorboard dans **Pytorch** dans ```torch.utils.ensorboard``` qui permet de faire le pont de pytorch vers cet outil. 

Le principe est le suivant :
* tensorboard fait tourner en fait un serveur web local qui va lire les fichiers de log dans un répertoire local. L'affichage se fait dans votre navigateur à partir d'un lien fourni lors du lancement de tensorboard.
* Les éléments que vous souhaitez visualiser (scalaire, graphes, distributions, histogrammes) sont écrits dans le fichier de log à partir d'un objet **SummaryWriter** .
* la méthode ```add_scalar(tag, valeur, global_step)``` permet de logger une valeur à un step donné, ```add_scalar(tag, tag_scalar_dic, global_step)``` un ensemble de valeurs par l'intermédiaire du dictionnaire ```tag_scalar_dic``` (un regroupement des scalaires est fait en fonction du tag passé, chaque sous-tag séparé par un **/**).

Il existe d'autres méthodes ```add_XXX``` pour visualiser par exemple des images, des histogrammes (cf <a href=https://pytorch.org/docs/stable/tensorboard.html>la doc </a>).

Le code suivant illustre une manière de l'utiliser. 

In [ ]:
# Spécial notebook, les commandes suivantes permettent de lancer tensorboard
# En dehors du notebook, il faut le lancer à la main dans le shell : 
# tensorboard --logdir logs
%load_ext tensorboard
%tensorboard --logdir /tmp/logs
from torch.utils.tensorboard import SummaryWriter
# Spécial notebook : pour avoir les courbes qui s'affichent dans le notebook, 
# sinon aller à l'adresse web local indiquée lors du lancement de tensorboard
from tensorboard import notebook
notebook.display() # A voir si vous avez une autre fenêtre de gestion de tensorboard ou si vous le voulez à la suite

EPS = 1e-5
EPOCHS=1000
netSeq = torch.nn.Sequential(torch.nn.Linear(Xdim,5),torch.nn.Tanh(),torch.nn.Linear(5,1))
netDeuxCouches = DeuxCouches()
netSeq.name = "Sequentiel" # nommer les modèles
netDeuxCouches.name = "DeuxCouches"


mseloss = torch.nn.MSELoss()
for model in [netSeq, netDeuxCouches]:
    ## Obtention d'un SummaryWriter
    ## meme répertoire que la commande %tensorboard --logdir logs 
    summary = SummaryWriter(f"/tmp/logs/test/{model.name}/") # on peut ajouter un timestamp ou des paramètres

    optim = torch.optim.SGD(params=model.parameters(),lr=EPS) 
    for i in range(EPOCHS):
        cumloss = 0
        for bx, by in loader:
            loss = mseloss(model(housing_x),housing_y.view(-1,1))
            optim.zero_grad()
            loss.backward()
            optim.step()  
            cumloss+= loss.item()
        summary.add_scalar(f"loss",cumloss,i) # c'est ici qu'on fait le lien


# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###